In [1]:
# -*- coding: utf-8 -*-

# multiple_regression.py

import sys
sys.path.append("../code-python3-ru")

from collections import Counter
from functools import partial
from lib.linear_algebra import dot, vector_add
from lib.statistics import median, standard_deviation
from lib.probability import normal_cdf
from lib.gradient_descent import minimize_stochastic
from lib.simple_linear_regression import total_sum_of_squares
import math, random

def predict(x_i, beta):
    return dot(x_i, beta)

def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

def squared_error(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

def squared_error_gradient(x_i, y_i, beta):
    """градиент (относительно beta), соответствующий i-ой квадратичной ошибке"""
    return [-2 * x_ij * error(x_i, y_i, beta)
            for x_ij in x_i]

def estimate_beta(x, y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error,
                               squared_error_gradient,
                               x, y,
                               beta_initial,
                               0.001)

def multiple_r_squared(x, y, beta):
    sum_of_squared_errors = sum(error(x_i, y_i, beta) ** 2
                                for x_i, y_i in zip(x, y))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(y)

def bootstrap_sample(data):
    """случайно отбирает len(data) элементов с возвратом (с повторами"""
    return [random.choice(data) for _ in data]

def bootstrap_statistic(data, stats_fn, num_samples):
    """оценивает функцию stats_fn на бустрап-выборках из данных
    в количестве num_samples"""
    return [stats_fn(bootstrap_sample(data))
            for _ in range(num_samples)]

def estimate_sample_beta(sample):
    x_sample, y_sample = list(zip(*sample)) # трюк с "разъединением" списка
    return estimate_beta(x_sample, y_sample)

def p_value(beta_hat_j, sigma_hat_j):
    if beta_hat_j > 0:
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)

#
# РЕГУЛЯРИЗОВАННАЯ РЕГРЕССИЯ
#

# alpha - это ГИПЕРПАРАМЕТР, контролирующий, насколько жестким будет штраф
# иногада его называют лямбда-параметром, но этот термин уже имеет свое
# значение в языке Python
def ridge_penalty(beta, alpha):
  return alpha * dot(beta[1:], beta[1:])

def squared_error_ridge(x_i, y_i, beta, alpha):
    """оценить ошибку плюс штраф для beta"""
    return error(x_i, y_i, beta) ** 2 + ridge_penalty(beta, alpha)

def ridge_penalty_gradient(beta, alpha):
    """градиент штрафа в гребневой регрессии"""
    return [0] + [2 * alpha * beta_j for beta_j in beta[1:]]

def squared_error_ridge_gradient(x_i, y_i, beta, alpha):
    """градиент, соответствующий i-ой квадратичной ошибке,
    включая гребневой штраф"""
    return vector_add(squared_error_gradient(x_i, y_i, beta),
                      ridge_penalty_gradient(beta, alpha))

def estimate_beta_ridge(x, y, alpha):
    """применить градиентный спуск для подгонки гребневой регрессии
    со штрафом alpha"""
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(partial(squared_error_ridge, alpha=alpha),
                               partial(squared_error_ridge_gradient,
                                       alpha=alpha),
                               x, y,
                               beta_initial,
                               0.001)

def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])

if __name__ == "__main__":

    x = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],
         [1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],
         [1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],
         [1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],
         [1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],
         [1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],
         [1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],
         [1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],
         [1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],
         [1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],
         [1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],
         [1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],
         [1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],
         [1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],
         [1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],
         [1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],
         [1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],
         [1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],
         [1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],
         [1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],
         [1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],
         [1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],
         [1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],
         [1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],
         [1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],
         [1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],
         [1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],
         [1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],
         [1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],
         [1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],
         [1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],
         [1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],
         [1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],
         [1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],
         [1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],
         [1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],
         [1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],
         [1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],
         [1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],
         [1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],
         [1,1,1,1],[1,1,1,1],[1,1,5,1]]

    daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,
                          54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,
                          35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,
                          20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,
                          27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,
                          26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,
                          30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,
                          19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,
                          28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,
                          26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,
                          14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,
                          18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,
                          36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,
                          19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,
                          36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,
                          32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,
                          13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,
                          19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,
                          31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,
                          21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]
    random.seed(0)
    beta = estimate_beta(x, daily_minutes_good) # [30.63, 0.972, -1.868, 0.911]
    print("beta", beta)
    print("r-квадрат", multiple_r_squared(x, daily_minutes_good, beta))
    print()

    print("отступление: бутстрапирование")
    # 101 точек, чьи значения очень близки к 100
    close_to_100 = [99.5 + random.random() for _ in range(101)]

    # 101 точек, из них 50 значений близки к 0, а другие 50 близки к 200
    far_from_100 = ([99.5 + random.random()] +
                    [random.random() for _ in range(50)] +
                    [200 + random.random() for _ in range(50)])

    print()
    print("bootstrap_statistic(close_to_100, median, 100):")
    print(bootstrap_statistic(close_to_100, median, 100))
    print()
    print("bootstrap_statistic(far_from_100, median, 100):")
    print(bootstrap_statistic(far_from_100, median, 100))
    print()

    random.seed(0) # чтобы получить повторимые результаты

    bootstrap_betas = bootstrap_statistic(list(zip(x, daily_minutes_good)),
                                          estimate_sample_beta,
                                          100)

    bootstrap_standard_errors = [
        standard_deviation([beta[i] for beta in bootstrap_betas])
        for i in range(4)]

    print()
    print("стандартные ошибки бутстрапированных выборок", bootstrap_standard_errors)
    print()

    print("p_value(30.63, 1.174)", p_value(30.63, 1.174))
    print("p_value(0.972, 0.079)", p_value(0.972, 0.079))
    print("p_value(-1.868, 0.131)", p_value(-1.868, 0.131))
    print("p_value(0.911, 0.990)", p_value(0.911, 0.990))
    print()

    print("регуляризация")
    print()
    
    random.seed(0)
    for alpha in [0.0, 0.01, 0.1, 1, 10]:
        beta = estimate_beta_ridge(x, daily_minutes_good, alpha=alpha)
        print("alpha", alpha)
        print("beta", beta)
        print("скалярное_произведение(beta[1:],beta[1:])", dot(beta[1:], beta[1:]))
        print("r-квадрат", multiple_r_squared(x, daily_minutes_good, beta))
        print()


beta [30.619881701311712, 0.9702056472470465, -1.8671913880379478, 0.9163711597955347]
r-квадрат 0.6800074955952597

отступление: бутстрапирование

bootstrap_statistic(close_to_100, median, 100):
[100.02634537429101, 100.06586584030461, 100.06850285070729, 100.06586584030461, 100.08022923777746, 100.02634537429101, 100.0251392489555, 100.06586584030461, 100.0251392489555, 100.18315165595742, 100.06627711829937, 100.06586584030461, 100.06627711829937, 100.06627711829937, 100.08022923777746, 100.08022923777746, 99.97928565593025, 99.95763666335068, 100.06627711829937, 99.95763666335068, 100.06586584030461, 100.06627711829937, 100.06850285070729, 100.08732646013036, 100.0340458696825, 100.02634537429101, 100.16675086620182, 100.11133261277095, 100.02634537429101, 100.0340458696825, 100.02048509159759, 100.06586584030461, 100.0340458696825, 100.02048509159759, 100.06850285070729, 100.06850285070729, 99.95763666335068, 100.08022923777746, 100.02048509159759, 100.07674830805912, 100.06586584